# IMPORT

In [2]:
from transformers import pipeline
import torch
import pandas as pd

if torch.cuda.is_available():
    print("GPU disponible !")
    print(f"Nom du GPU : {torch.cuda.get_device_name(0)}")
else:
    print("GPU non disponible, utilisation du CPU.")

c:\Users\Fournier\Documents\Cours CS\fillière recherche\psychological_analysis_of_breast_cancer_patients\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU non disponible, utilisation du CPU.


# MODEL

In [3]:
distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    device="cpu",
    return_all_scores=True
)

Device set to use cpu
c:\Users\Fournier\Documents\Cours CS\fillière recherche\psychological_analysis_of_breast_cancer_patients\.venv\lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
total_params = sum(p.numel() for p in distilled_student_sentiment_classifier.model.parameters())
print(f"Nombre total de poids : {total_params}")

Nombre total de poids : 135326979


## Test model

In [5]:
# english test
distilled_student_sentiment_classifier ("I love this movie and i would watch it again and again!")

[[{'label': 'positive', 'score': 0.9731044769287109},
  {'label': 'neutral', 'score': 0.01691008359193802},
  {'label': 'negative', 'score': 0.00998548325151205}]]

In [6]:
distilled_student_sentiment_classifier("What is your chemo regimen? People ice for different reasons - some for nail issues and some for neuropathy prevention.  I am less convinced about the neuropathy prevention as that is more of a nervous system issue which I am not sure localized icing can prevent, and I took some supplements to accomplish that.  I iced because taxanes are known for nail lifting and loss. I used bags of frozen peas and brought them myself in a small cooler.  Nobody else was icing, but I didn't care. I started icing 10 minutes before the Taxotere was started, and continued until 10 minutes afterward. I placed a bag of frozen peas over each foot on the top so it had contact with my toenails, and then held two bags of frozen peas in my lap on a beach towel and dug my fingernails in so the peas covered the fingernails. My oncologist was aware that I was icing - didn't have any opinion regarding effectiveness, my center did not offer icing supplies or assistance, but many centers do.")

[[{'label': 'positive', 'score': 0.12064313888549805},
  {'label': 'neutral', 'score': 0.3123035132884979},
  {'label': 'negative', 'score': 0.5670533180236816}]]

# IMPORT DATA

In [7]:
topics_name = "Chemotherapy - Before, During, and After"
df = pd.read_csv("../data/scraped_data_in_treatment/" + topics_name + ".csv")

In [8]:
dico_thread_conversation = df.groupby('thread_title')['comment_text'].apply(list).to_dict()
print(next(iter(dico_thread_conversation)))
print(next(iter(dico_thread_conversation.values()))[1])

1St Chemo Fulphila Injection
Hi @sukhbrar -Sorry to hear your Mom is going through chemo. Nice that you can be her support person.Unfortunately, diarrhea and/or constipation is very common with chemo. Everyone is different, but when I was on TC, the first week was the worst, the 2nd week I started feeling better and the 3rd week I felt pretty good, just in time to get ready for my next treatment. 🙄If you want to read more about people's experiences with TC, there is a thread devoted to it here:
    
        https://community.breastcancer.org/en/discussion/697783/anyone-on-just-taxotere-and-cytoxan
    
Anyone on just Taxotere and Cytoxan?DianeBJan 2, 2008I just started a regimen of taxotere and cytoxan for four rounds. I really need to talk to someone who has been on just this regimen. My first session did not go well. It dropped my sodium levels too low and I had a couple of seizures. They kept me in the hospital for two nights. I have been achey and weak since.  Does it get better? I

In [9]:
def get_table_score_from_output(output_model):
    table_score = {}
    all_dico = output_model[0]
    for dico in all_dico:
        emotion = dico['label']
        table_score[emotion] =dico['score']

    return table_score



In [10]:
print(get_table_score_from_output([[{'label': 'positive', 'score': 0.12064313888549805},
  {'label': 'neutral', 'score': 0.3123035132884979},
  {'label': 'negative', 'score': 0.5670533180236816}]]))

{'positive': 0.12064313888549805, 'neutral': 0.3123035132884979, 'negative': 0.5670533180236816}


In [ ]:
def get_binary_sentiment_per_thread(dico_thread_conv):
    for thread_name, conversation in dico_thread_conv.items():
        for nbr_st,sentence in enumerate(conversation):
            # output_one_sentence = distilled_student_sentiment_classifier(sentence)
            try:
                output_one_sentence = distilled_student_sentiment_classifier(sentence)
            except:
                pass
            score_one_sentence = get_table_score_from_output(output_one_sentence)
            if nbr_st==0:
                score_thread = score_one_sentence
            else:
                score_thread['positive']+= score_one_sentence['positive']
                score_thread['neutral']+= score_one_sentence['neutral']
                score_thread['negative']+= score_one_sentence['negative']

        #normalisation
        score_thread['positive'] = score_thread['positive']/len(conversation)
        score_thread['neutral'] = score_thread['neutral']/len(conversation)
        score_thread['negative'] = score_thread['negative']/len(conversation)

        print()
        print(f"\nName thread : {thread_name}")
        print(f"Score : {score_thread}")
        # break
        

get_binary_sentiment_per_thread(dico_thread_conversation)



name thread : 1St Chemo Fulphila Injection
score : {'positive': 0.3228397397324443, 'neutral': 0.18445654725655913, 'negative': 0.4927037201123312}


name thread : 2 Years Out From Chemo Neuropathy Worse
score : {'positive': 0.3264806896448135, 'neutral': 0.1956542257219553, 'negative': 0.477865095436573}


name thread : Ac Any Non Horror Stories Tips
score : {'positive': 0.3533850941807032, 'neutral': 0.2042571570724249, 'negative': 0.44235775284469125}


name thread : Ac Done Terrified Of T
score : {'positive': 0.3147894710302353, 'neutral': 0.20528771355748177, 'negative': 0.47992281466722486}


name thread : Ac Infusion And Bladder Irritation
score : {'positive': 0.3291606584356891, 'neutral': 0.21676504239439964, 'negative': 0.45407430910401875}


name thread : Ac Or Cmf
score : {'positive': 0.4851195976138115, 'neutral': 0.18080840036273002, 'negative': 0.3340720169246197}


name thread : Ac T Latest Advice
score : {'positive': 0.30701096914708614, 'neutral': 0.2171191032975912